# Imports

In [2]:
#Read JSON files
import json

# Data Exploration

## Import Events file
Import the open data file 

In [5]:
with open("../Data/Original/Evenementen.json") as event_data:
    events = json.load(event_data)

## Example contents
Below is an example snippet of an event in the file

In [6]:
#Return eventname
print("Event: ", events[0]["title"])

#Return exact location event in human terms
print("City: ", events[0]["location"]["city"])
print("Adress: ", events[0]["location"]["adress"])
print("Zipcode: ", events[0]["location"]["zipcode"])

#Check source crowdedness
print("Coordinates: ", events[0]["location"]["latitude"], events[0]["location"]["longitude"]) 

#Check whether the data has an event
print("Date: ", events[0]["dates"])

Event:  Springsnow Festival
City:  AMSTERDAM
Adress:  Diverse locaties / Various locations
Zipcode:  1012 JS
Coordinates:  52,3726380 4,8941060
Date:  {'startdate': '20-04-2018', 'enddate': '20-05-2018'}


## Subset
As not all the variables are usable, we make a subset of the following variables:
- *Event Name*: Show the event name
- *Coordinates*: Show the coordinates of the event
- *Data*: Show the event date(s)

In [9]:
#Save all the events in the list
events_list = []

#Loop over all the events
for event in events: 
    #Save the coordinates in a dict, as these are seperate in original file
    coordinates = {"Latitude": event["location"]["latitude"], "Longtitude": event["location"]["longitude"]}

    #Dict with all data single event
    event_dict = {"Event": event["title"], "Coordinates": coordinates, "Data": event["dates"]}
    
    #Append dict to list
    events_list.append(event_dict)

In [10]:
#Example snippet
print(events_list[0])

{'Event': 'Springsnow Festival', 'Coordinates': {'Latitude': '52,3726380', 'Longtitude': '4,8941060'}, 'Data': {'startdate': '20-04-2018', 'enddate': '20-05-2018'}}


## Save to file
Save the list of subset events to file

In [11]:
with open("../Data/Modified/Events.json", "a") as writer:
    json.dump(events_list, writer)

In [ ]:
## Check
Check whether the file is correctly saved

In [4]:
with open("../Data/Modified/Events.json") as f:
    events = json.load(f)

JSONDecodeError: Extra data: line 1 column 165 (char 164)